In [1]:
import pandas as pd
from sklearn.model_selection import KFold
from catboost import CatBoostClassifier
import numpy as np
from sklearn.metrics import f1_score
import pickle


In [2]:
dataset=pd.read_csv('C:\\Users\\HP-PC\\LOAN CATEGORY PREDICTION\\DATA\\train.csv')


In [3]:
dataset[dataset['Interest_Rate']==1]

,Loan_ID,Loan_Amount_Requested,Length_Employed,Home_Owner,Annual_Income,Income_Verified,Purpose_Of_Loan,Debt_To_Income,Inquiries_Last_6Mo,Months_Since_Deliquency,Number_Open_Accounts,Total_Accounts,Gender,Interest_Rate
0,10000001,"7,000",< 1 year,Rent,68000.0,not verified,car,18.37,0,NaN,9,14,Female,1
4,10000005,"17,000",8 years,Own,96000.0,VERIFIED - income source,debt_consolidation,22.17,1,NaN,19,30,Female,1
9,10000010,"30,000",10+ years,Own,163000.0,VERIFIED - income,debt_consolidation,11.13,0,NaN,24,43,Male,1
20,10000021,"4,000",10+ years,Rent,55000.0,not verified,credit_card,23.04,0,10.0,15,21,Female,1
25,10000026,"2,200",7 years,Mortgage,82000.0,not verified,major_purchase,5.02,1,NaN,14,21,Female,1
30,10000031,"10,075",9 years,Mortgage,65520.0,VERIFIED - income,debt_consolidation,23.96,1,22.0,11,31,Female,1
32,10000033,"12,000",2 years,Mortgage,75000.0,VERIFIED - income source,debt_consolidation,9.20,1,NaN,10,25,Female,1
33,10000034,"8,875",4 years,NaN,NaN,VERIFIED - income,debt_consolidation,24.32,0,NaN,5,8,Male,1
37,10000038,"21,000",2 years,NaN,130000.0,VERIFIED - income,credit_card,5.25,0,64.0,8,19,Female,1
39,10000040,"8,400",10+ years,NaN,95000.0,VERIFIED - income,debt_consolidation,17.32,0,NaN,13,41,Male,1


In [3]:
data={'< 1 year':1, '4 years':4, '7 years':7, '8 years':8, '2 years':2,
       '10+ years':10, '1 year':1, '6 years':6, '9 years':9, '3 years':3,
       '5 years':5}

dataset['Length_Employed']=dataset['Length_Employed'].map(data)
dataset['Length_Employed']=dataset['Length_Employed'].fillna(0)

In [4]:
dataset['Home_Owner']=dataset['Home_Owner'].fillna('Other')

In [5]:
dataset['Months_Since_Deliquency']=dataset['Months_Since_Deliquency'].fillna(0)


In [6]:
dataset['Annual_Income']=dataset['Annual_Income'].fillna(dataset['Annual_Income'].median())

In [7]:
dataset["Loan_Amount_Requested"] = dataset["Loan_Amount_Requested"].str.replace(",", "")

In [8]:
dataset.head()

,Loan_ID,Loan_Amount_Requested,Length_Employed,Home_Owner,Annual_Income,Income_Verified,Purpose_Of_Loan,Debt_To_Income,Inquiries_Last_6Mo,Months_Since_Deliquency,Number_Open_Accounts,Total_Accounts,Gender,Interest_Rate
0,10000001,7000,1.0,Rent,68000.0,not verified,car,18.37,0,0.0,9,14,Female,1
1,10000002,30000,4.0,Mortgage,63000.0,VERIFIED - income,debt_consolidation,14.93,0,17.0,12,24,Female,3
2,10000003,24725,7.0,Mortgage,75566.4,VERIFIED - income source,debt_consolidation,15.88,0,0.0,12,16,Male,3
3,10000004,16000,1.0,Other,56160.0,VERIFIED - income source,debt_consolidation,14.34,3,0.0,16,22,Male,3
4,10000005,17000,8.0,Own,96000.0,VERIFIED - income source,debt_consolidation,22.17,1,0.0,19,30,Female,1


In [13]:
x=dataset.drop(['Loan_ID','Interest_Rate'],axis=1).values
y=dataset['Interest_Rate']

In [18]:
#using catboost
kfold, scores = KFold(n_splits=5, shuffle=True, random_state=0), list()
for train, test in kfold.split(x):
    x_train, x_test = x[train], x[test]
    y_train, y_test = y[train], y[test]
    model = CatBoostClassifier(random_state=27, max_depth=5, n_estimators=100, verbose=500)
    model.fit(x_train, y_train, cat_features=[ 2, 4, 5, 11])
    preds = model.predict(x_test)
    score = f1_score(y_test, preds, average="weighted")
    scores.append(score)
    print(score)
print("Average: ", sum(scores)/len(scores))

Learning rate set to 0.5
0:	learn: 1.0357264	total: 126ms	remaining: 12.5s
99:	learn: 0.9081827	total: 11.3s	remaining: 0us
0.5309230147411327
Learning rate set to 0.5
0:	learn: 1.0351669	total: 133ms	remaining: 13.2s
99:	learn: 0.9071188	total: 11.5s	remaining: 0us
0.5291740821824357
Learning rate set to 0.5
0:	learn: 1.0349073	total: 124ms	remaining: 12.3s
99:	learn: 0.9071879	total: 11.7s	remaining: 0us
0.5301525531903198
Learning rate set to 0.5
0:	learn: 1.0354564	total: 294ms	remaining: 29.1s
99:	learn: 0.9077124	total: 17.3s	remaining: 0us
0.5312845237960951
Learning rate set to 0.5
0:	learn: 1.0388075	total: 266ms	remaining: 26.3s
99:	learn: 0.9073161	total: 13.8s	remaining: 0us
0.5328242710219436
Average:  0.5308716889863854


In [20]:
pickle.dump(model,open('model.pkl','wb')) 